# DIP_Mechanics
# Mathematical modeling <a class="anchor" id="mathematicalModel"></a>

Cartesian coordinates of the $leg_1$ and pole_{2} will be 
\begin{equation}
\begin{bmatrix}
q_1 \\ q_2 
\end{bmatrix}
\qquad \text{and} \qquad 
\begin{bmatrix}
q_1 + l \sin(q_2) \\ l \cos(q_2)] 
\end{bmatrix}
\end{equation}
where ($q_1$) is the horizontal position of the cart and ($q_2$) is the angle of the pendulum.

The state, $x$ and state derivative, $\dot{x}$ will give the dynamics of the system in first-order form and represnets the minimal generalized coordinates and its derivatives
\begin{equation}
 x =
 \begin{bmatrix}
  q_1 \\ q_2 \\ \dot{q_1} \\ \dot{q_2}
\end{bmatrix}	
\qquad \text{and} \qquad 		
\dot{x} = f ( x , u ) =
\begin{bmatrix}
\dot{q_1} \\ \dot{q_2} \\ \ddot{q_1} \\ \ddot{q_2}
\end{bmatrix}
\end{equation}

Using Lagrangian method, 	
$ L = T - V $
where T is the kinetic energy and V the potential energy of the system. L summarizes the system dynamics.

\begin{align}
& T = T_{cart} + T_{pole} \\
&T_{cart} = \frac{1}{2} m_1\dot{ q_1}^2 \\
&T_{pole} = \frac{1}{2} m_2  [\dot{(q_1 + l \sin(q_2)}^2 +\dot{(l\cos(q_2)}^2] \\\\
& V = V_{cart} + V_{pole} \\
&V = 0 + m_2 g l \cos(q_2) \\
\end{align}

The potential energy of the cart is zero since it is not moving vertically and it has a height of 0.

Thus $L = \frac{1}{2} m_1 q_1^2 + \frac{1}{2} m_2 [\dot{(x + l \sin(q_2)}^2 +\dot{(l\cos(q_2)}^2] - m_2 g l \cos(q_2)$


The generalized coordinates could be defined as 
\begin{equation}	
X = 
\begin{bmatrix}
q_1  \\ q_2 
\end{bmatrix}
\end{equation}
To yield the equations of motion for the cart/pendulum system, the Lagrangian equations will: 
\begin{align}
&\frac{d}{dt}(\frac{\partial L}{\partial{\dot{q_1}}}) - \frac{\partial L}{\partial{\dot{q_1}}} = u \\
&\frac{d}{dt}(\frac{\partial L}{\partial{\dot{q_2}}}) - \frac{\partial L}{\partial{\dot{q_2}}} = 0
\end{align}

Solving for the partial and time derivatives will yield
\begin{align}
&(m_1 + m_2) \ddot{q_1} + m_1 l \ddot{q_2} \cos(q_2) - m_2 l \dot{q_2}^2 \sin(q_2) = u \\
&m_2 l \ddot{q_1} \cos(q_2) + m_2 l^2 \ddot{q_2}  - m_2 g l \sin(q_2) = 0 
\end{align}
The detailed derivations of the derivatives can be found on this <a href=\"http://www.aoengr.com/Dynamics/PendulumOnCart.pdf\">webisite.</a> 

Rearranging the above equations and solving for the $\ddot{q_1}$ and $\ddot{q_2} $ terms respectively,
\begin{align}
&\ddot{q_1} = \frac{ l m_2 \sin(q_2) \dot{q_2}^2 + u + m_{2} g \cos(q_{2}) sin(q_2)} {m_1 + m_2\sin^2(q_2)} \\
&\ddot{q_2} = \frac{ l m_2 \cos(q_2) \sin(q_2^2) \dot{q_2}^2 + u \cos(q_2) + (m_1 + m_2) g \sin(q_2)} {l (m_1 + m_2\sin^2(q_2))}
\end{align}


In [164]:
using JuMP, Ipopt

function qDDTwoLinkManipulator(z)
    m = Model(solver = IpoptSolver(print_level=0))
    # assign joint displacements / velocities from state variables
    @variable(m, q[1:4])
    @variable(m, D[1:2,1:2])
    @variable(m, G[1:2,1])
    @variable(m, V[1:2,1])
    @variable(m, B[1:2,1])
#     @variable(m, dummy[1:4])
    @constraint(m, q .== z)
    # model parameters
    g = 9.81
    m1 = 1
    m2 = 1
    L1 = 10
    L2 = 10
    Lc1 = L1/2
    Lc2 = L2/2
    I1 = (1/12)*m1*L1^2
    I2 = (1/12)*m2*L2^2;

    # precalculate sin and cos terms
    @NLexpression(m, s, sin(q[1]))
    @NLexpression(m, c1, cos(q[1]))
    @NLexpression(m, s2, sin(q[2]))
    @NLexpression(m, c2, cos(q[2]))
    @NLexpression(m, s12, sin(q[1]+q[2]))
    @NLexpression(m, c12, cos(q[1]+q[2]))
    

    # mass matrix calculation
    @NLconstraint(m, D[1,1] == m1*Lc1^2 + m2*(L1^2 + Lc2^2 + 2*L1*Lc2*c2) + I1 + I2)
    @NLconstraint(m, D[1,2] == m2*Lc2^2+ L1*Lc2*c2 + I2)
#     @NLexpression(m, d21, d12)
     @NLconstraint(m, D[2,1] == m2*Lc2^2+ L1*Lc2*c2 + I2)
    @NLconstraint(m, D[2,2] == m2*Lc2^2 + I2)

    #coriolis and centrifugal terms
    @NLexpression(m, h, -m2*L1*Lc2*s2)
    @NLexpression(m, c111, 0)
    @NLexpression(m, c121, h)
    @NLexpression(m, c211, h)
    @NLexpression(m, c221, h)
    @NLexpression(m, c112, -h)
    @NLexpression(m, c122, 0)
    @NLexpression(m, c222, 0)
    @NLconstraint(m, V[1,1] == c121*q[3]*q[4] + c211*q[4]*q[3] + c221*q[4]^2)
    @NLconstraint(m, V[2,1] == c112*q[3]^2)
     
    # gravity vector
    @NLexpression(m, g1, (m1*Lc1 + m2*L1)*g*c1 + m2*Lc2*g*c12)
    @NLexpression(m, g2, m2*Lc2*g*c12)
    @NLconstraint(m, G[1,1] == g1)
    @NLconstraint(m, G[1,1] == g2)

    #joint viscous damping (added to bring manipulator to rest)
    @variable(m, b == 0.0)
    @NLexpression(m, B[1,1] == b*q[3])
    @NLexpression(m, B[2,1] == b*q[4])
    
    #input torque
    @variable(m, T[1:2,1])
#     @constraint(m, T[1,1]==0)
#     @constraint(m, T[2,1]==0)
     
    #form joint acceleration vector 
    @variable(m, qd[1:2,1])
    @variable(m, Dq1[1:2,1])
    @variable(m, Dq2[1:2,1])
#     @NLconstraint(m, qd == D\(-V - G - B + T))
    
    @NLconstraint(m, Dq1 == D[1,:]'*qd)
    @NLconstraint(m, Dq1 == D[2,:]'*qd)
    @NLconstraint(m, Dq[1,:] + V[1,1] + G[1,1] + B[1,1] == T[1,1])
    @NLconstraint(m, Dq[2,:] + V[2,1] + G[2,1] + B[2,1] == T[2,1])
    
    # min Torque
    @objective(m, Min, sum(T.^2))
#     @objective(m, Min, sum(q+dummy))
    
    solve(m)
#     stuff = getvalue(q)
#     print(stuff)

    # assign state variable derivatives
    zDot[1,1] = getvalue(q[3])
    zDot[2,1] = getvalue(q[4])
    zDot[3,1] = getvalue(qd[1])
    zDot[4,1] = getvalue(qd[2])
    return zDot
end

qDDTwoLinkManipulator (generic function with 1 method)

In [165]:
dT = .03      #integration step size
tend = 0.09     #simulation run time    
numPts = floor(tend/dT)
z = zeros(4,numPts)    #pre-allocate array memory (to improve sim. speed)
# dq = zeros(2,numPts)
t = zeros(1,numPts)
z[:,1] = [pi/2; 0; .1; 1] #initial position and velocity
       
 
# integrate equations of motion
# integrationMethod = 3;
for i = 1:numPts-1
# for i = 1:2
#     if integrationMethod == 1
#         # Euler:
#         qDot = qDDTwoLinkManipulator(q);
#         q = q + dT*qDot
#     elseif integrationMethod == 2
#         # Improved Euler (Huen):
#         k1 = qDDTwoLinkManipulator(q)
#         k2 = qDDTwoLinkManipulator(q + k1*dT)
#         q = q + 0.5*(k1 + k2)*dT
#     elseif integrationMethod == 3
        # Runge-Kutta 4th order
#         print(q)
        k1 = qDDTwoLinkManipulator(z)
#         print(k1)
#         k2 = qDDTwoLinkManipulator(q + 0.5*k1*dT)
#         k3 = qDDTwoLinkManipulator(q + 0.5*k2*dT)
#         k4 = qDDTwoLinkManipulator(q + k3*dT)
#         q = q + (1/6)*(k1 + 2*k2 + 2*k3 + k4)*dT
#     end
    # store joint position and velocity for rendering/plotting
#     z[:,i+1] = q
#     t[1,i+1] = t[1,i] + dT;        
end

LoadError: [91mMethodError: no method matching parseNLExpr_runtime(::JuMP.Model, ::JuMP.JuMPArray{JuMP.Variable,2,Tuple{UnitRange{Int64},Int64}}, ::Array{ReverseDiffSparse.NodeData,1}, ::Int64, ::Array{Float64,1})[0m
Closest candidates are:
  parseNLExpr_runtime(::JuMP.Model, [91m::Number[39m, ::Any, ::Any, ::Any) at C:\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\Users\the1k\AppData\Local\JuliaPro-0.6.2.1\pkgs-0.6.2.1\v0.6\JuMP\src\parsenlp.jl:196
  parseNLExpr_runtime(::JuMP.Model, [91m::JuMP.Variable[39m, ::Any, ::Any, ::Any) at C:\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\Users\the1k\AppData\Local\JuliaPro-0.6.2.1\pkgs-0.6.2.1\v0.6\JuMP\src\parsenlp.jl:202
  parseNLExpr_runtime(::JuMP.Model, [91m::JuMP.NonlinearExpression[39m, ::Any, ::Any, ::Any) at C:\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\\Users\the1k\AppData\Local\JuliaPro-0.6.2.1\pkgs-0.6.2.1\v0.6\JuMP\src\parsenlp.jl:208
  ...[39m

In [37]:
k1

LoadError: [91mUndefVarError: k1 not defined[39m

In [32]:
#-----------------------------------------------------------------------
#  EXAMPLE NUMERICAL INTEGRATION OF DYNAMIC EQUATIONS
#-----------------------------------------------------------------------
# clear all; close all; clc;

# initialize integration variables    
dT = .03      #integration step size
tend = 10      #simulation run time    
numPts = floor(tend/dT)
q = zeros(4,numPts)    #pre-allocate array memory (to improve sim. speed)
dq = zeros(2,numPts)
t = zeros(1,numPts)
q[:,1] = [pi/2; 0; .1; 1] #initial position and velocity
       
 
# integrate equations of motion
integrationMethod = 3;
# for i = 1:numPts-1
for i = 1:2
    if integrationMethod == 1
        # Euler:
        qDot = qDDTwoLinkManipulator(q);
        q = q + dT*qDot
    elseif integrationMethod == 2
        # Improved Euler (Huen):
        k1 = qDDTwoLinkManipulator(q)
        k2 = qDDTwoLinkManipulator(q + k1*dT)
        q = q + 0.5*(k1 + k2)*dT
    elseif integrationMethod == 3
        # Runge-Kutta 4th order
        k1 = qDDTwoLinkManipulator(q)
        k2 = qDDTwoLinkManipulator(q + 0.5*k1*dT)
        k3 = qDDTwoLinkManipulator(q + 0.5*k2*dT)
        k4 = qDDTwoLinkManipulator(q + k3*dT)
        q = q + (1/6)*(k1 + 2*k2 + 2*k3 + k4)*dT
    end
    # store joint position and velocity for rendering/plotting
    z[:,i+1] = q
    t[1,i+1] = t[1,i] + dT;        
end

# %-----------------------------------------------------------------------
# %  RENDERING INITIALIZATION
# %-----------------------------------------------------------------------

# %----set rendering window view parameters
# L1 = 10;  L2 = 10;
# f_handle = 1;
# axis_limits = L1*[-2 2 -2 2 -1 1];
# render_view = [1 .5 2]; view_up = [0 1 0];
# SetRenderingViewParameters(axis_limits,render_view,...
#                            view_up,f_handle); 
# camproj perspective % turns on perspective

# %----initialize rendering

# % link 1 rendering initialization
# r1 = L1/10; sides1 = 10; axis1 = 1; norm_L1 = 1.0;
#     linkColor1 = [0 0.75 0]; plotFrame1 = 0;
# d1 =  CreateLinkRendering(L1,r1,sides1,axis1,norm_L1,linkColor1,...
#                           plotFrame1,f_handle);

# % link 2 rendering initialization
# r2 = L2/10; sides2 = 10; axis2 = 1; norm_L2 = 1.0;
#     linkColor2 = [0.75 0 0]; plotFrame2 = 0;
# d2 =  CreateLinkRendering(L2,r2,sides2,axis2,norm_L2,linkColor2,...
#                           plotFrame2,f_handle);

# %-------------------------------------------------------------------
# %  DISPLAY INTERATION RESULTS
# %-------------------------------------------------------------------
# for i = 1:numPts
#     % Update frame {1}
#     c = cos(q(1,i)); s = sin(q(1,i)); L = L1;
#     T10 = [c  -s  0  L*c
#            s   c  0  L*s
#            0   0  1   0
#            0   0  0   1];
#     % Update frame {2}
#     c = cos(q(2,i)); s = sin(q(2,i)); L = L2;
#     T21 = [c  -s  0  L*c
#            s   c  0  L*s
#            0   0  1   0
#            0   0  0   1];
#     T20 = T10*T21;
#     UpdateLink(d1,T10);
#     UpdateLink(d2,T20);
#     if i == 1;  %pause at start of simulation rendering
#         pause;
#     end
#     pause(dT);
# end            

LoadError: [91mMethodError: no method matching *(::Float64, ::JuMP.NonlinearExpression)[0m
Closest candidates are:
  *(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424
  *(::Float64, [91m::Float64[39m) at float.jl:379
  *(::Real, [91m::Complex{Bool}[39m) at complex.jl:254
  ...[39m